In [1]:
import pandas as pd
import numpy as np
from difflib import SequenceMatcher

In [2]:
def similar(a, b):
    grades = []
    a = a.split(' ')
    b = b.split(' ')
    for p1 in a:
        for p2 in b:
            grades.append(SequenceMatcher(None, p1, p2).ratio())
    return max(grades)

def similar_bairro(a,b):
    return SequenceMatcher(None, a, b).ratio()

In [3]:
df_socio = pd.read_excel('BF_Dados_Socio_Economicos.xlsx')

In [4]:
df_socio['atividade'] = df_socio['atividade'].fillna('ATIVIDADE NÃO INFORMADA').apply(str).apply(lambda x: 'ATIVIDADE NÃO INFORMADA' if len(x) < 3 else x)

In [5]:
def transform_str(x):
    x = x.split('/')[0]
    x = x.split(' E ')[0]
    x = x.replace(' DE ',' ').replace(' EM ',' ').replace(' (A)','')
    x = ' '.join(x.split(' ')[:3])
    return x
df_socio['atividade'] = df_socio['atividade'].apply(transform_str)

In [6]:
atv = list(df_socio['atividade'].unique())
len(atv)

7832

In [7]:
group = df_socio[['identificação','atividade']].groupby('atividade').count().sort_values('identificação',ascending = False)
top100 = group.index[:210]
top100

Index(['DIARISTA', 'PEDREIRO', 'VENDAS', 'AUTONOMO', 'VENDEDOR', 'AGRICULTURA',
       'VENDEDORA', 'COSTUREIRA', 'AGRICULTOR', 'MANICURE',
       ...
       'TRANSPORTES', 'SECRETARIA', 'MOTORISTA APP', 'PET SHOP',
       'MARKETING DIGITAL', 'VENDAS LINGERIE', 'POUSADA', 'PIZZAIOLO',
       'LOCATÁRIO', 'SERVIÇOS RURAIS'],
      dtype='object', name='atividade', length=210)

In [8]:
mapa = {}
for atividade in top100:
    for value in mapa.values():
        if similar(atividade,value) > 0.7:
            mapa[atividade] = value
            break
    if atividade not in mapa:
        mapa[atividade] = atividade

In [9]:
newtop = set(mapa.values())
len(newtop)

100

In [10]:
cash = {}
def convert(x):
    if x in cash:
        return cash[x]
    for atv in newtop:
        if similar(atv,x) > 0.7:
            cash[x] = atv
            return atv
    cash[x] = 'OUTROS'
    return 'OUTROS'
for valor in df_socio['atividade'].unique():
    convert(valor)

In [11]:
df_socio['atividade'] = df_socio['atividade'].apply(lambda x: convert(x))

In [12]:
group = df_socio[['identificação','atividade']].groupby('atividade').count().sort_values('identificação',ascending = False)
group

,identificação
atividade,
OUTROS,6071
VENDEDOR,4951
VENDAS,4674
DIARISTA,4589
AUTONOMO,3197
...,...
TORNEARIA,62
EMPREENDEDOR INDIVIDUAL,60
BRECHO,58


In [13]:
df_socio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63825 entries, 0 to 63824
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   identificação                 63713 non-null  object 
 1   valor_solicitado              63713 non-null  object 
 2   prazo_em_meses                63697 non-null  object 
 3   melhor_data_vencimento        63727 non-null  object 
 4   melhor_valor_parcela          63697 non-null  object 
 5   atividade                     63825 non-null  object 
 6   tipo_atividade                48788 non-null  object 
 7   tempo_atividade               55418 non-null  object 
 8   tempo_atuacao_ramo_atividade  48763 non-null  object 
 9   total_receitas                63635 non-null  object 
 10  numero_de_pessoas_na_casa     44073 non-null  float64
 11  situacao_do_imovel            44074 non-null  object 
 12  tempo_de_residancia__anos     44068 non-null  object 
 13  m

In [14]:
df = df_socio.copy()

In [15]:
def convert_to_int(x):
    try:
        return int(x)
    except:
        return None
def convert_to_float(x):
    try:
        return float(x)
    except:
        return None
def convert_to_string(x,coluna):
    if not x:
        return f'{coluna} Não Informado'
    if type(x) != str:
        return f'{coluna} Não Informado'
    return x

In [16]:
df['identificação'] = df['identificação'].apply(convert_to_int)
df = df.loc[~df['identificação'].isna()]

In [17]:
df['valor_solicitado'] = df['valor_solicitado'].apply(convert_to_float)

In [18]:
df['prazo_em_meses'] = df['prazo_em_meses'].apply(convert_to_int)

In [19]:
df['melhor_data_vencimento'] = df['melhor_data_vencimento'].apply(convert_to_int)

In [20]:
df['melhor_valor_parcela'] = df['melhor_valor_parcela'].apply(convert_to_float)

In [21]:
df['atividade'] = pd.Categorical(df['atividade'])

In [22]:
df['tipo_atividade'] = df['tipo_atividade'].apply(lambda x: convert_to_string(x,'Tipo Atividade'))
df['tipo_atividade'] = pd.Categorical(df['tipo_atividade'])

In [23]:
df['tempo_atividade'] = df['tempo_atividade'].apply(lambda x: convert_to_string(x,'Tempo Atividade'))
df['tempo_atividade'] = pd.Categorical(df['tempo_atividade'])

In [24]:
df['tempo_atuacao_ramo_atividade'] = df['tempo_atuacao_ramo_atividade'].apply(lambda x: convert_to_string(x,'Tempo Ramo Atividade'))
df['tempo_atuacao_ramo_atividade'] = pd.Categorical(df['tempo_atuacao_ramo_atividade'])

In [30]:
df['total_receitas'] = df['total_receitas'].apply(convert_to_float)

In [25]:
df['situacao_do_imovel'] = df['situacao_do_imovel'].apply(lambda x: convert_to_string(x,'Situação Imóvel'))
df['situacao_do_imovel'] = pd.Categorical(df['situacao_do_imovel'])

In [26]:
df['tempo_de_residancia__anos'] = df['tempo_de_residancia__anos'].apply(lambda x: convert_to_string(x,'Tempo Residência'))
df['tempo_de_residancia__anos'] = pd.Categorical(df['tempo_de_residancia__anos'])

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63676 entries, 0 to 63675
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype   
---  ------                        --------------  -----   
 0   identificação                 63676 non-null  float64 
 1   valor_solicitado              63663 non-null  float64 
 2   prazo_em_meses                63653 non-null  float64 
 3   melhor_data_vencimento        62363 non-null  float64 
 4   melhor_valor_parcela          63660 non-null  float64 
 5   atividade                     63676 non-null  category
 6   tipo_atividade                63676 non-null  category
 7   tempo_atividade               63676 non-null  category
 8   tempo_atuacao_ramo_atividade  63676 non-null  category
 9   total_receitas                63583 non-null  float64 
 10  numero_de_pessoas_na_casa     44033 non-null  float64 
 11  situacao_do_imovel            63676 non-null  category
 12  tempo_de_residancia__anos     63676 non-null  

In [32]:
df = df.reset_index(drop = True)

In [33]:
df.to_feather('BF_Dados_Socio_Economicos.feather')